In [0]:
from collections import *
def char_hist(data):
  lm = Counter();
  for i in range(len(data)):
    lm[data[i]] += 1;
  s = float(sum(lm.values()));
  hist = {c:cnt/s for c, cnt in lm.items()};
  return hist;

def train_char_lm(data, order=5):
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in range(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm
  
def train_char_lm_smooth(data,order=2,add_k=1):
   lm = defaultdict(Counter);
   pad = "~"*order;
   data = pad + data;
   length = len(list(set(data)));
   for i in range(len(data) - order):
      history, char = data[i:i+order], data[i+order]
      lm[history][char] += 1;
   def normalize(counter,length):
      s = float(sum(counter.values()));
      return [(c,(cnt+add_k)/(s + add_k * length)) for c,cnt in counter.items()];
   outlm = {hist:normalize(chars,length) for hist, chars in lm.items()}
   return outlm    

def calculate_prob_with_backoff(char, history, lms, lambdas, len_char=65):
  length = len(lambdas);
  prob = 0;
  for i in range(1,length+1):
    if i == 1 and char in lms[0]:
      prob += lms[0][char] * lambdas[0];
      continue;
    if not history[-(i-1):] in lms[i-1]:
       continue;
    next_char = lms[i-1][history[-(i-1):]];
    for j in range(len(next_char)):
        ch = next_char[j][0];
        if ch == char:
           prob += next_char[j][1] * lambdas[i-1];
           break;
  if (prob == 0):
      prob = 1.0/len_char;
  return prob;

In [3]:
!pip install unidecode
import unidecode
import string
#file = unidecode.unidecode(open('tiny-shakespeare.txt').read());
file = unidecode.unidecode(open('enwik8.txt','r',encoding='utf-16').read());
#lm = train_char_lm_smooth(file, order=1)
hist = char_hist(file);
lm = train_char_lm(file,order=5);
perplex_tr = perplexity(lm,file,order=5);
'''
lms = [];
lms.append(hist);
order = 5;
for i in range(1,order + 1):
  lm = train_char_lm(file,order=i);
  lms.append(lm);
print(len(list(set(file))));
lambdas=[0.01,0.04,0.05,0.15,0.15,0.6];
lm = lms[order];
from random import random
#print(lm);
#lm_backoff = mix_lm_backoff(file,lms,lambdas);
perplex_tr = perplexity(lm,file,order=5);
print(perplex_tr)
'''

    100% |████████████████████████████████| 245kB 4.8MB/s 


NameError: ignored

In [0]:
import math
import random
def generate_letter(lm, history, order):
        history = history[-order:]
        if not history in lm:
           return 
        dist = lm[history]
        x = random.random();
        for c,v in dist:
            x = x - v;
            if x <= 0: return c
        return dist[math.floor(random.random()*len(dist))][0];
def generate_text(lm, order, nletters=1000):
    history = "~" * order; 
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

In [0]:
#print(generate_text(lm, 3));
#file = unidecode.unidecode(open('shakespeare_sonnets.txt').read());
#file2 = unidecode.unidecode(open('nytimes_article.txt').read());
def perplexity(lm, data, order=1, add_k = 0.5, length = 65):
  N = len(data) - order;
  pad = "~" * order;
  test = pad + data;
  perplex = 1;
  for i in range(N):
    prob = 0;
    history, char = data[i:i+order], data[i+order];
    if not history in lm:
      prob = 1.0 / length;
    else:
     for c, v in lm[history]:
       if c == char:
         prob = v;
         break;
       prob = float(add_k) / length;
    perplex *= (1.0/prob)**(1/N);
  return perplex;

def perplexity_backoff(lm, lms, data, lambdas, order=3, length = 65):
  N = len(data) - order;
  pad = "~" * order;
  test = pad + data;
  perplex = 1;
  for i in range(N):
     history, char = data[i:i+order], data[i+order];
     prob = calculate_prob_with_backoff(char, history, lms, lambdas);
     perplex *= (1.0/prob)**(1/N);
  return perplex;

#perplex_backoff = perplexity_backoff(lm, lms, file2, lambdas, order=5);
#perplex = perplexity(lm, file2, order=5);
#print(perplex_backoff, perplex);

In [0]:
for i in range(len(lm_backoff["First"])):
         print(lm_backoff["First"][i])

0


Prefix "though":
(' ', 0.44074074074074077)
('t', 0.5407407407407407)
('\n', 0.011111111111111112)
(',', 0.003703703703703704)
('!', 0.003703703703703704)

In [15]:
print(generate_text(lm,5,10000));

~~~ k = 0 is the be shape time institutional [[Depth-first [[Theophiles]]
| {{IPA|/S/}}
|-align=center&quot;O Lord is locating many civilisation:''
74.96 birth and 400 member 13]], [[United ''Andronovo (record on languages were he publisher ===
{{seealso|List of readings II of English through this academy Awards elected to current provide and non and keep slowed Alders (2 Samuel]]).

On [[Danube, the executive===
:''Main article: [[North touchess====
[[Image:Ampered enterests are stormingham]], during pairs Middle of Script]]-based on the &quot;rightarrow gauge:''
79.09 year open range &quot;bissextidi'' [{{IPA|/ p@ /}}||[[Khazardous. Citta degree on the merely on the [[Office of Bates]] 
* [http://angelientReference source of a model of the State age of ten to video games is differed image-museum]]
* [[Anne Approach in series, which is 70th is standard one of negation to provements]] + [CA,B]
*[http://www.apples on married AC designs and the path of its claim to states, conversions) i